# PCA Troubleshooting

This Jupyter Notebook runs regressions for MOSAIKS features with PCA applied. 

Right now, this notebook only runs such regressions for the "population" label.

### Imports

In [8]:
import numpy as np
import pandas as pd
import dill
import numpy as np

from mosaiks.code.mosaiks import config as cfg
from mosaiks.code.mosaiks.utils import io
from mosaiks.code.mosaiks.solve import data_parser as parse
from mosaiks.code.mosaiks.solve import solve_functions as solve
from mosaiks.code.mosaiks.solve import interpret_results as ir
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

### Setup
This notebook is only running regressions for the label "population" for now.

#### Run for MOSAIKS features

In [19]:
X, latlons= io.get_X_latlon(cfg, "UAR")

#### Run for TorchGeo RCF features

In [ ]:
with open("data/int/feature_matrices/CONTUS_UAR_torchgeo.pkl", "rb") as f:
        arrs = dill.load(f)
X = pd.DataFrame(
    arrs["X"].astype(np.float64),
    index=arrs["ids_X"],
    columns=["X_" + str(i) for i in range(arrs["X"].shape[1])],
)

# get latlons
latlons = pd.DataFrame(arrs["latlon"], index=arrs["ids_X"], columns=["lat", "lon"])

# sort both
latlons = latlons.sort_values(["lat", "lon"], ascending=[False, True])
X = X.reindex(latlons.index)

In [20]:
label = "population"

solver = solve.ridge_regression

In [21]:
#Test all lambdas (specified in config file)
this_lambdas = io.get_lambdas(cfg, label, best_lambda_fpath=None)

c = io.get_filepaths(cfg, label)
c_app = getattr(c, label)

#Bounds from config file
if c_app["logged"]:
    bounds = np.array([c_app["us_bounds_log_pred"]])
else:
    bounds = np.array([c_app["us_bounds_pred"]])

### Merge and Transform Data

In [22]:
(
    this_X,
    this_X_test,
    this_Y,
    this_Y_test,
    this_latlons,
    this_latlons_test,
    this_emb,
    this_emb_test
) = parse.merge_dropna_transform_split_train_test(
    c, label, X, latlons, None
)

Loading labels...
Merging labels and features...
Splitting training/test...


#### Remove training points so size of train set is divisible by 5 (for Cross-validation)

In [23]:
n = len(this_X)
this_X = this_X[:-(n%5)]
this_Y = this_Y[:-(n%5)]
this_latlons = this_latlons[:-(n%5)]

### PCA

1) Combine Train and Test (``this_X``, ``this_X_test``)
2) Initialize PCA (PCA(0.99) retaines 99% of the variance; can also specify PCA(num_components = 13) or other desired number)
3) ``fit_transform`` to combined data
4) Split data into train and test

PCA is done at this step since when X is initially retrieved as a data frame, some values are NaN

#### First combine train and test so PCA can be applied to all

In [24]:
X = np.concatenate((this_X, this_X_test), axis=0)

#### Run the next block to scale the data

In [15]:
scaling = StandardScaler()
scaling.fit(X)
X = scaling.transform(X)

#### Perform PCA (to modify number of components, change PCA(n_components) to be either specified number of components or % retained variance

In [25]:
#PCA
print("Performing PCA...")
pca = PCA(0.99)
X_pca = pca.fit_transform(X)

print("Number of PCA Components: ", pca.n_components_)

#Separate train and test
this_X = X_pca[:this_X.shape[0], :]
this_X_test = X_pca[this_X.shape[0]:, :]

### Sanity Check

Number of test samples: 13585

1) For "population", number of train samples is 54343 (this is reduced to 54340 for 5-fold cross validation)

In [26]:
print(this_X.shape)
print(this_X_test.shape)

(54340, 8192)
(13585, 8192)


### Run Regressions

Code from MOSAIKS run_regressions.ipynb file

In [27]:
print("*** Running regressions for: {label}".format(label=label))

print("Training model...")
import time

st_train = time.time()
kfold_results = solve.kfold_solve(
    this_X,
    this_Y,
    solve_function=solver,
    num_folds=c.ml_model["n_folds"],
    return_model=True,
    lambdas=this_lambdas,
    return_preds=True,
    svd_solve=False,
    clip_bounds=bounds,
)
print("")

# get timing
training_time = time.time() - st_train
print("Training time:", training_time)

## Store the metrics and the predictions from the best performing model
best_lambda_idx, best_metrics, best_preds = ir.interpret_kfold_results(
    kfold_results, "r2_score", hps=[("lambdas", c_app["lambdas"])]
)
best_lambda = this_lambdas[best_lambda_idx][0]

# Get test set predictions
st_test = time.time()
holdout_results = solve.single_solve(
    this_X,
    this_X_test,
    this_Y,
    this_Y_test,
    lambdas=best_lambda,
    svd_solve=False,
    return_preds=True,
    return_model=False,
    clip_bounds=bounds,
)

#Get timing
test_time = time.time() - st_test
print("Test set training time:", test_time)

print("R^2 score: ", holdout_results["metrics_test"][0][0][0]["r2_score"])


*** Running regressions for: population
Training model...
on fold (of 5): 1 2 3 4 5 
Training time: 95.62686777114868
Test set training time: 17.239691972732544
R^2 score:  0.7413341635475623
